### PWM -> Position weight matrix. A matrix where eaach entry corresponds to one base of a motif assigned with the frequency found in the motif.

### For example if a transcription factor recognizes a motif of 8 nucleotides the a PWM could be the following
![image.png](attachment:image.png)

### PSSM -> Position Speciic Scoring Matrix
### Is a PWM that takes into acount the background frequency of the nucleotides
Each entry on the PSSM is the log2 ratio of the frequency of the nucleotide in the motif and the background frequency of the nucleotide
![image.png](attachment:image.png)

## Shannon Entropy
![image.png](attachment:image.png)

 Entropy maximizes when the all possibilities are equally likely to occure.
 Shannon Information is the difference of the initial entropy and the final entropy
 This information is measured in bits

The most dificult is when I do not have any information about the motif and I am trying to discover it
For denovo motif discovery there are approaches with randomized algorithms(Gibbs sampling) and comparative genomicd approaches.

Comparative genomics approach -> cross-species sequence alignment to find phylogenetic footprints, regions of unusually high conservation.

In [ ]:
gata = ['TTATAG', 'AGATAT', 'AGATAG', 'ATATCT', 'AGATAG', 'AGATAG', 'AGATAG', 'TGATAA', 'AGATAA',
 'AGATAA', 'CGATAG', 'AGAGTT', 'TGATAA', 'TGATAA', 'AGATGG', 'AGATAG', 'AGATTG', 'AGATAA', 'TGATAA',
  'AGATAA', 'AGATAG', 'TGATAG', 'TGATCA', 'TTATCA', 'AGATGG', 'TGATAT', 'AGATAG', 'TGATAA', 'GGATAC',
   'AGATAA', 'CGATAA', 'TGATAG', 'AGATAA', 'TGATTA', 'AGATAA', 'AGATAG', 'TGATAT', 'AGATAA', 'TCAGAG',
    'AAGTAG', 'AGATTA', 'TGATAG', 'TGATAG', 'AGATAC', 'TGATTG', 'AGATTA', 'AGAATA', 'AGATAA', 'AGATTA',
     'AGCTTC']


In [ ]:
len(gata)

50

okur

A function to create a pwm from a list of aligned sequences

In [ ]:
def pwm(sequnce_list):
    nuc = ['A', 'T', 'G', 'C']
    profile=[[0 for i in range(len(sequnce_list[0]))] for j in range(len(nuc))] #Two list comprehensions
    #The first one returns a list of zeros with length equal to the length of each sequence.
    #The second one returns 4 lists of length equal to the length of the sequence
    #return profile
    #
    for instance in sequnce_list: #iterates through the sequences
        for j in range(len(instance)): #iterates through the indexes of the nucleotides of each sequence
            residue = instance[j] #sets the residue variable
            profile[nuc.index(residue)][j] +=1 #Adds one to the residue value in the nested list.
            profile[nuc.index(residue)][j]=float(profile[nuc.index(residue)][j]) #Converts the previously added value to float
    #return profile
    import numpy as np
    pwm = np.array(profile)
    pwm = pwm/len(sequnce_list)
    return pwm

In [ ]:
print(pwm(sequnce_list=gata))

[[0.6  0.02 0.96 0.02 0.72 0.44]
 [0.34 0.06 0.   0.94 0.18 0.1 ]
 [0.02 0.9  0.02 0.04 0.04 0.4 ]
 [0.04 0.02 0.02 0.   0.06 0.06]]


In [ ]:
#Write a function that reads a fasta sequence
def readfasta(fastafile):
    import regex as re
    f=open(fastafile,'r')
    seq = ''
    total = 0
    for line in f:
        x = re.match('>', line)
        if x == None:
            length = len(line)
            total+=length
            seq+=line[0:length-1]
    return seq


In [ ]:
#Write a function that calculates nucleotiude frewuences
def nuccomp(sequence):
    import numpy as np
    nucfreq = [0,0,0,0]
    nuc = ['A', 'T', 'G', 'C']
    for i in range(len(nuc)):
        nucfreq[i] = sequence.count(nuc[i])
    nucfreq = np.array(nucfreq)/len(sequence)
    return nucfreq

In [ ]:
seq = readfasta('./Staaur.fa')
my_pwm = pwm(sequnce_list=gata)
nuc_freq = nuccomp(seq)

In [ ]:
nuc_freq

array([0.33585935, 0.33495328, 0.16384496, 0.16534241])

Now that I have computed both nucleotide frwuences and the PWM I can combine them to build a PSSM


In [ ]:
def pssm(pwm, nucfreqs):
    import numpy as np
    import math
    pseudocount = 0.01 #This is very important because if iwant to compute the frequency of any sequence i want to avoid having zeros
    pssm = [[0 for i in range(len(pwm[0]))] for j in range(len(nucfreqs))] #This line creates an empty 4 by len(sequence) pssm
    for i in range(len(nucfreqs)): #The range is from 1 to 4
        pssm[i]=(pwm[i]+pseudocount)/nucfreqs[i] #pssm[i] is the ith row of the pssm array
        #The above line takes each row of the pwm matrix and devides it with the nuc_freq array
    for i in range(len(pssm)):
        for k in range(len(pssm[0])):
            pssm[i][k] = math.log(pssm[i][k]/math.log(2))


    return np.array(pssm)

In [ ]:
my_pssm = pssm(my_pwm, nuc_freq)

In [ ]:
my_pssm

array([[ 0.96327942, -2.04898216,  1.42711653, -2.04898216,  1.14286499,
         0.65906804],
       [ 0.41045501, -1.1989829 , -3.14489305,  1.40898384, -0.20045407,
        -0.74699777],
       [-1.33121032,  2.0810369 , -1.33121032, -0.8203847 , -0.8203847 ,
         1.28374946],
       [-0.82948262, -1.34030824, -1.34030824, -2.43892053, -0.49301038,
        -0.49301038]])

In [ ]:
import numpy as np
np.sum(my_pssm, axis = 1)


array([ 0.09436466, -3.47188893, -0.93840367, -6.9350404 ])

In [ ]:
import math
math.log(1,2)

0.0

In [ ]:
my_pssm[:,1]

array([-2.04898216, -1.1989829 ,  2.0810369 , -1.34030824])

Now that I have also computed a PSSM I can write a function to search in any sequence using this specific PSSM to look for motifs. The probability of any sequence is the producto of the frequnces at each position of the sequence. I have to set a score threshold for that.


In [ ]:
def pssmSearch(pssm, sequence, threshold):
    nuc = ['A', 'T', 'G', 'C']
    hits = []
    instances = []
    x = [] #A list that holds indexes for the second step
    allscores = [] #for plotting reasons

    # Step 1: Calculation of the maximum possible pssm score
    maxPssm = 0
    for j in range(len(pssm[0])): #This essentially gives me the lenth of each sequence.
        maxPssm = maxPssm + max(pssm[:,j])
        #This means add to max the maximum value of the jth column pssm[:,j] is the jth column of the pssm array
        #Note that although I am computing probability I am using the sum of the pssm values and not their product.
        #This is because at the last step of the pssm function I log transformed the frequences

    # Step 2: Search
    for i in range(len(sequence) - len(pssm[0])):
        x.append(i)
        instance = sequence[i:i+len(pssm[0])]
        score = 0
        for l in range(len(instance)):
            score = score + pssm[nuc.index(instance[l])][l]
            if (score > threshold*maxPssm):
                hits.append(i)
                instances.append(instance)
                allscores.append(score)
    return (hits, instances, allscores)


In [ ]:
pssmSearch(my_pssm, seq, 0.80)[1]


NameError: name 'my_pssm' is not defined

In [ ]:
def pwmEntropyInformation(pwm):
    import numpy as np
    k = pwm.shape[1] #This gives me the number of columns(So the length of the sequence represented in the pwm)
    information = np.zeros([1,k]) #Computing the information in each  position
    for i in range(k):
        information[0,i] = 2 - abs(sum([elem*np.log2(elem) for elem in pwm[:,i] if elem > 0])) #This formula is explained on the top of the document.
        sumInfo = np.sum(information)
        scaledSumInfo = sumInfo/k

    return (information, sumInfo, scaledSumInfo)


In [ ]:
import numpy as np
H_init = 2
H_final = 0
for i in my_pwm[:,0]:
    H_final += i*np.log2(i)

print(H_init - abs(H_final))


0.7300155336155016


In [ ]:
my_pwm


array([[0.6 , 0.02, 0.96, 0.02, 0.72, 0.44],
       [0.34, 0.06, 0.  , 0.94, 0.18, 0.1 ],
       [0.02, 0.9 , 0.02, 0.04, 0.04, 0.4 ],
       [0.04, 0.02, 0.02, 0.  , 0.06, 0.06]])

In [ ]:
pwmEntropyInformation(my_pwm)

(array([[0.73001553, 1.39390935, 1.71770781, 1.61745733, 0.78417406,
         0.37435552]]),
 6.617619603779856,
 1.102936600629976)

In [ ]:
len(pwmsearches)
len(pssmsearches)

3336

In [ ]:
import numpy as np
np.zeros([1,2])

array([[0., 0.]])

# Gibbs Sampling

Problem -> Given p strings and a length k, find the most mutually similar length-k substring from each string
Gibbs sampling replaces a single k-mer at each iteration and thus moves with more caution in the space of all motifs.
“Gibbs sampling” is the basis behind a general class of algorithms
that is a type of local search.


Gibbs Sampling do not gurantees good performance but often works well in practice

Assumes that we know the length of the motif we are looking for
Assumes that each input sequence contains exactly 1 real instance of the motif

Gibbs Sampling Algorithm


• Initialization: select random locations in sequences x1, …, xN, and compute an initial
model M from these locations

I divid all sequences to kmers
I take one kmer from each sequence and create a pwm
I scan all the sequences with the pwm and I substitute the chosen kmer with the one that best fits the pwm
Update the PWM
Repeat until reaching a set threshold

In [ ]:
for i in range(10):
    print(i)


0
1
2
3
4
5
6
7
8
9
